### Task 1
Leia o arquivo de texto ​ wordcount.txt, ​ e conte as palavras que contém até 10 letras. Conte também quantas palavras com mais de 10 letras existem no texto.
Dataset: ​ https://storage.googleapis.com/luizalabs-hiring-test/wordcount.txt 
        
Exemplo do dataset final:

[('two', 2),

('behold', 1),

('itself', 3),

(‘MAIORES QUE 10’, 3)]

#### 0. Baixar arquivo para local

In [19]:
!mkdir -p dataset
!wget https://storage.googleapis.com/luizalabs-hiring-test/wordcount.txt
!mv wordcount.txt dataset/wordcount.txt
!ls dataset/*

--2020-11-11 21:41:20--  https://storage.googleapis.com/luizalabs-hiring-test/wordcount.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 2800:3f0:4004:801::2010, 2800:3f0:4004:800::2010, 2800:3f0:4004:808::2010, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|2800:3f0:4004:801::2010|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18436 (18K) [text/plain]
Saving to: ‘wordcount.txt’

wordcount.txt       100%[===================>]  18.00K  --.-KB/s    in 0.03s   

2020-11-11 21:41:20 (575 KB/s) - ‘wordcount.txt’ saved [18436/18436]

dataset/wordcount.txt


#### 1. Importando bibliotecas

In [9]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import split
from pyspark.sql.functions import explode
from pyspark.sql.functions import desc
from pyspark.sql.functions import col
from pyspark.sql.functions import length
from pyspark.sql.functions import trim
from pyspark.sql.functions import lower

#### 2. Criando session

In [10]:
spark = SparkSession.builder \
    .master("local") \
    .appName("Task 1") \
    .getOrCreate()

#### 3. Lendo arquivo .txt

In [11]:
def removePunctuation(column):
    return trim(lower(regexp_replace(column, '[^\sa-zA-Z0-9]', ''))).alias('value')

In [12]:
df = spark.read.text("dataset/wordcount.txt").select(removePunctuation(col('value')))

In [13]:
df.count()

15

In [14]:
df.show(15, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#### 4. Quantas linhas?

In [15]:
df.count()

15

In [16]:
df.show()

+--------------------+
|               value|
+--------------------+
|hendrerit montes ...|
|                    |
|aliquam facilisis...|
|                    |
|scelerisque dui h...|
|                    |
|nascetur mollis p...|
|                    |
|aptent malesuada ...|
|                    |
|justo congue nibh...|
|                    |
|volutpat mi erat ...|
|                    |
|tincidunt element...|
+--------------------+



#### 5. Nome e tipo da coluna?

In [17]:
df.dtypes

[('value', 'string')]

#### 6. Vendo primeiro registro completo

In [18]:
df.first()

Row(value='hendrerit montes purus luctus dictum est mattis est phasellus dignissim rhoncus cubilia sit nunc at interdum vivamus luctus ante ac amet elementum congue justo eget egestas vitae per lacus sociis purus quisque luctus eleifend inceptos condimentum etiam vel augue vivamus proin eget placerat conubia odio inceptos feugiat etiam venenatis class lacus ultrices convallis aenean diam donec in sociis neque ac dignissim at ultrices nisl semper netus cubilia nunc ante porttitor aliquet proin eros viverra sodales netus integer auctor tortor ultricies cubilia netus posuere ipsum quam montes sociosqu vivamus condimentum in lectus nascetur eu etiam nec tempor posuere sagittis rutrum luctus facilisis feugiat turpis platea ut eros porttitor viverra ligula ipsum etiam placerat elementum sem semper nec fringilla dictumst eget sit malesuada sagittis arcu justo cum odio ante lectus potenti nibh amet vel netus penatibus vivamus rutrum orci sollicitudin commodo placerat nulla morbi pulvinar conub

#### 7. Words

In [19]:
dfWords = (df
           .select(explode(split(df.value, ' ')).alias('word'))
           .where(col('word') != ''))

In [20]:
df.show()

+--------------------+
|               value|
+--------------------+
|hendrerit montes ...|
|                    |
|aliquam facilisis...|
|                    |
|scelerisque dui h...|
|                    |
|nascetur mollis p...|
|                    |
|aptent malesuada ...|
|                    |
|justo congue nibh...|
|                    |
|volutpat mi erat ...|
|                    |
|tincidunt element...|
+--------------------+



In [21]:
df.count()

15

In [22]:
dfWords.show()

+---------+
|     word|
+---------+
|hendrerit|
|   montes|
|    purus|
|   luctus|
|   dictum|
|      est|
|   mattis|
|      est|
|phasellus|
|dignissim|
|  rhoncus|
|  cubilia|
|      sit|
|     nunc|
|       at|
| interdum|
|  vivamus|
|   luctus|
|     ante|
|       ac|
+---------+
only showing top 20 rows



In [24]:
dfWords.count()

2500

In [25]:
def wordCount(wordListDF):
    """Creates a DataFrame with word counts.

    Args:
        wordListDF (DataFrame of str): A DataFrame consisting of one string column called 'word'.

    Returns:
        DataFrame of (str, int): A DataFrame containing 'word' and 'count' columns.
    """
    return wordListDF.groupBy('word').count()

In [132]:
# topWordsAndCountsDF = wordCount(dfWords).orderBy("count", ascending=False)

In [26]:
# topWordsAndCountsDF.show()

In [27]:
# len(topWordsAndCountsDF.first()['word'])

In [28]:
dfLength = dfWords.withColumn("length", length("word"))

In [29]:
dfLength = dfWords.filter(length(col('word')) <= 10)

In [30]:
dfLength.show()

+---------+
|     word|
+---------+
|hendrerit|
|   montes|
|    purus|
|   luctus|
|   dictum|
|      est|
|   mattis|
|      est|
|phasellus|
|dignissim|
|  rhoncus|
|  cubilia|
|      sit|
|     nunc|
|       at|
| interdum|
|  vivamus|
|   luctus|
|     ante|
|       ac|
+---------+
only showing top 20 rows



In [31]:
topWordsAndCountsDF = wordCount(dfLength).orderBy("count", ascending=False)

In [33]:
print(topWordsAndCountsDF.show())

+---------+-----+
|     word|count|
+---------+-----+
| nascetur|   25|
|   montes|   22|
|  laoreet|   22|
|  vivamus|   22|
|    etiam|   21|
|    purus|   21|
|hendrerit|   21|
| praesent|   21|
|  quisque|   21|
| habitant|   21|
|     diam|   20|
|fermentum|   20|
|   tempus|   20|
|   tellus|   20|
|     odio|   20|
|     urna|   19|
|      sit|   19|
|        a|   19|
|    curae|   18|
|     arcu|   18|
+---------+-----+
only showing top 20 rows

None


In [174]:
dfLength = dfWords.filter(length(col('word')) > 10)

In [175]:
dfLength.count()

115

In [177]:
newRow = spark.createDataFrame([["MAIORES QUE 10", dfLength.count()]])

In [178]:
newRow.show()

+--------------+---+
|            _1| _2|
+--------------+---+
|MAIORES QUE 10|115|
+--------------+---+



In [179]:
appended = topWordsAndCountsDF.union(newRow)


In [181]:
appended.collect()

[Row(word='nascetur', count=25),
 Row(word='laoreet', count=22),
 Row(word='vivamus', count=22),
 Row(word='montes', count=22),
 Row(word='hendrerit', count=21),
 Row(word='quisque', count=21),
 Row(word='praesent', count=21),
 Row(word='purus', count=21),
 Row(word='etiam', count=21),
 Row(word='habitant', count=21),
 Row(word='odio', count=20),
 Row(word='fermentum', count=20),
 Row(word='diam', count=20),
 Row(word='tempus', count=20),
 Row(word='tellus', count=20),
 Row(word='sit', count=19),
 Row(word='urna', count=19),
 Row(word='a', count=19),
 Row(word='curae', count=18),
 Row(word='interdum', count=18),
 Row(word='placerat', count=18),
 Row(word='arcu', count=18),
 Row(word='ante', count=18),
 Row(word='rhoncus', count=18),
 Row(word='vel', count=18),
 Row(word='inceptos', count=18),
 Row(word='venenatis', count=18),
 Row(word='sapien', count=18),
 Row(word='orci', count=18),
 Row(word='duis', count=18),
 Row(word='porttitor', count=17),
 Row(word='netus', count=17),
 Row(word

In [184]:
appended.count()

180

In [188]:
appended.repartition(1).write.csv('result/task1', mode='overwrite')